In [379]:
import math

class Value:
    def __init__(self,data,_children=(),_op='',label =''):
        self.data = data
        self.grad = 0.0
        self._backward = lambda:None
        self._prev = set(_children)
        self._op = _op
        self.label = label
    
    def __add__(self,other): 
        other = other if isinstance(other,Value) else Value(other)
        out = Value(self.data+other.data,(self,other),"+")
        def _backward():
            self.grad += out.grad
            other.grad += out.grad
        out._backward = _backward
        return out
    
        
    def __mul__(self,other):
        other = other if isinstance(other,Value) else Value(other)
        out = Value(self.data*other.data,(self,other),"*")
        def _backward():
            self.grad += other.data*out.grad
            other.grad += self.data*out.grad
        out._backward = _backward
        return out
        
    def __pow__(self,other):
        assert isinstance(other,(int,float)), "only support int or float" 
        out = Value(self.data**other,(self,),f"**{other}")
        def _backward():
            self.grad += other*self.data**(other-1)*out.grad
        self._backward = _backward
        return out 
    
    def backward(self):
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)
        self.grad = 1.0
        for v in reversed(topo):
            v._backward()
    
    def tanh(self):
        x = self.data
        t = (math.exp(2*x) - 1)  / (math.exp(2*x) + 1)
        out = Value(t,(self,),label="tanh")
        def _backward():
            self.grad += (1 - t**2)*out.grad
        out._backward = _backward
        return out
    
    def exp(self):
        x = self.data
        out = Value(math.exp(x),(self,),label="exp")
        def _backward():
            self.grad += out.grad * out.data
        out._backward = _backward
        return out
    
    def __neg__(self): # -self
        return self * -1

    def __radd__(self, other): # other + self
        return self + other

    def __sub__(self, other): # self - other
        return self + (-other)

    def __rsub__(self, other): # other - self
        return other + (-self)

    def __rmul__(self, other): # other * self
        return self * other

    def __truediv__(self, other): # self / other
        return self * other**-1

    def __rtruediv__(self, other): # other / self
        return other * self**-1

    def __repr__(self):
        return f"Value(data={self.data}, grad={self.grad})"
    
    
    

In [380]:
import graphviz

In [381]:
def trace(root):
    nodes, edges = set(), set()
    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v._prev:
                edges.add((child,v))
                build(child)
    build(root)
    return nodes, edges

In [382]:
def draw_dot(root):
    g = graphviz.Digraph('gradient',format="svg",graph_attr={'rankdir':'LR'})
    nodes, edges = trace(root)
    for n in nodes:
        uid =str(id(n))
        g.node(name=uid,label="%s | data %.4f | grad %.4f" % (n.label,n.data,n.grad),shape='record')
        if n._op:
            g.node(name=uid+n._op,label=n._op)
            g.edge(uid+n._op,uid)
    
    for n1 , n2 in edges:
        g.edge(str(id(n1)),str(id(n2)) + n2._op)
    
    return g

In [383]:
x1 = Value(2.0,label='x1')
x2 = Value(0.0,label='x2')
w1 = Value(-3.0,label='w1')
w2 = Value(1.0,label='w2')
b = Value(6.88137358,label='b')
x1w1 = x1*w1 ; x1w1.label = 'x1w1' 
x2w2 = x2*w2 ; x2w2.label = 'x2w2'
x1w1x2w2 = x1w1+x2w2 ; x1w1x2w2.label = 'x1w1x2w2'
n = x1w1x2w2 + b ; n.label = 'n'
o = n.tanh() ; o.label = 'o'
o.backward()


In [384]:
print(o.data)
print(w1.grad)
print(w2.grad)
print(x1.grad)  
print(x2.grad)

0.707106777676776
1.000000009927133
0.0
-1.5000000148906993
0.5000000049635664


In [385]:
## doing the same thing but using pytorch

import torch

x1 = torch.tensor([2.0],requires_grad=True)
x2 = torch.tensor([0.0],requires_grad=True)
w1 = torch.tensor([-3.0],requires_grad=True)
w2 = torch.tensor([1.0],requires_grad=True)

b = torch.tensor([6.88137358],requires_grad=True)

n = x1*w1 + x2*w2 + b 
o = torch.tanh(n)
o.backward()

In [386]:
print(o.data.item())
print(w1.grad.item())
print(w2.grad.item())
print(x1.grad.item())  
print(x2.grad.item())



0.7071067094802856
1.000000238418579
0.0
-1.5000003576278687
0.5000001192092896


In [426]:
import random
class Neuron:
    def __init__(self,ninputs):
        self.w = [Value(random.uniform(-1,1)) for _ in range(ninputs)]
        self.b = Value(random.uniform(-1,1))
    
    def __call__(self,x):
        act = sum((xi*wi for xi,wi in zip(x,self.w)), self.b)
        out = act.tanh()
        return out
    
    def parameters(self):
        return self.w + [self.b]
    
class Layer:
    def __init__(self,ninputs,nneurons):
        self.neurons = [Neuron(ninputs) for _ in range(nneurons)]
    
    def __call__(self,x):
        outs =  [n(x) for n in self.neurons]
        return outs[0] if len(outs) == 1 else outs
    
    def parameters(self):
        return [p for n in self.neurons for p in n.parameters()]

class MLP:
    def __init__(self,ninputs,noutputs):
        sz = [ninputs] + noutputs
        self.layers = [Layer(sz[i],sz[i+1]) for i in range(len(noutputs))]

    def __call__(self,x):
        for layer in self.layers:
            x = layer(x)
        return x
    
    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]

x = [Value(random.uniform(-1,1)) for _ in range(3)]
n = MLP(3,[4,4,1])
n(x)

Value(data=0.811869989130927, grad=0.0)

In [427]:
#draw_dot(n(x))

In [428]:
xs = [
    [1.0,1.0,-1.0],
    [1.0,1.0,0.5],
    [0.0,1.0,-1.0],
    [-1.0,0.0,0.5]
]

ys = [1.0,1.0,0.0,-1.0] # desired targets 

In [434]:
y_pred = [n(x) for x in xs]
y_pred

[Value(data=0.8426014519190288, grad=0.0),
 Value(data=0.8849531967696909, grad=0.0),
 Value(data=0.8288850182613603, grad=0.0),
 Value(data=0.8036168824388453, grad=0.0)]

In [435]:
y_pred = [n(x) for x in xs]
loss = sum( (yout-ygt)**2 for ygt,yout in zip(ys,y_pred))
loss

Value(data=3.978094301988067, grad=0.0)

In [436]:
loss.backward()

In [437]:
n.layers[0].neurons[0].w[0].data

0.13952319783590106

In [438]:
n.layers[0].neurons[0].w[0].grad

0.0

In [439]:
for p in n.parameters():
    p.data -= 0.01 * p.grad

In [440]:
y_pred

[Value(data=0.8426014519190288, grad=0.0),
 Value(data=0.8849531967696909, grad=0.0),
 Value(data=0.8288850182613603, grad=0.0),
 Value(data=0.8036168824388453, grad=0.0)]

In [441]:
n.parameters()[40].grad

0.0